# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
import pickle

In [2]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table("Message", con=engine)

Overview of the dataframe

In [3]:
df.head()

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
2,2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df["message"]
Y = df[df.columns[5:]]

In [5]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
     # tokenize
    tokens = word_tokenize(text)
     # initiate Lemmatizer
    lemmatizer = WordNetLemmatizer()
    # Lemmatize 
    cleanedTokens = []
    for token in tokens:
        clean_token = lemmatizer.lemmatize(token).lower().strip()
        cleanedTokens.append(clean_token)
    return cleanedTokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ahmad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ahmad\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size = 0.3)

In [10]:
# train classifier
pipeline.fit(X_train.as_matrix(), y_train.as_matrix())

c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\ahmad\ap

c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
def perf_report(model, X_test, y_test):
    '''
    Function to generate classification report on the model
    Input: Model, test set ie X_test & y_test
    Output: Prints the Classification report
    '''
    y_pred = model.predict(X_test)
    for i, col in enumerate(y_test):
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]))

In [12]:
perf_report(pipeline, X_test, y_test)

related
              precision    recall  f1-score   support

           0       0.60      0.34      0.44      1813
           1       0.82      0.93      0.87      5995
           2       0.42      0.18      0.25        57

    accuracy                           0.79      7865
   macro avg       0.61      0.48      0.52      7865
weighted avg       0.77      0.79      0.77      7865

request
              precision    recall  f1-score   support

           0       0.88      0.98      0.93      6493
           1       0.83      0.36      0.50      1372

    accuracy                           0.88      7865
   macro avg       0.85      0.67      0.71      7865
weighted avg       0.87      0.88      0.85      7865

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7838
           1       0.00      0.00      0.00        27

    accuracy                           1.00      7865
   macro avg       0.50      0.50      0.50      786

c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7727
           1       0.50      0.01      0.01       138

    accuracy                           0.98      7865
   macro avg       0.74      0.50      0.50      7865
weighted avg       0.97      0.98      0.97      7865

military
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      7597
           1       0.47      0.08      0.14       268

    accuracy                           0.97      7865
   macro avg       0.72      0.54      0.56      7865
weighted avg       0.95      0.97      0.95      7865

child_alone
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7865

    accuracy                           1.00      7865
   macro avg       1.00      1.00      1.00      7865
weighted avg       1.00      1.00      1.00      7865

water
              precision    recall  f1-score   

              precision    recall  f1-score   support

           0       0.85      0.98      0.91      6285
           1       0.81      0.30      0.43      1580

    accuracy                           0.84      7865
   macro avg       0.83      0.64      0.67      7865
weighted avg       0.84      0.84      0.81      7865



### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
parameters = {'clf__estimator__max_depth': [10, 50, None],
              'clf__estimator__min_samples_leaf':[2, 5, 10]}

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
cv.fit(X_train.as_matrix(), y_train.as_matrix())
perf_report(cv, X_test, y_test)

c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.

c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "

c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "

c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "

c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "

...

c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "

related
              precision    recall  f1-score   support

           0       0.67      0.21      0.31      1813
           1       0.80      0.97      0.87      5995
           2       1.00      0.11      0.19        57

    accuracy                           0.79      7865
   macro avg       0.82      0.43      0.46      7865
weighted avg       0.77      0.79      0.74      7865

request
              precision    recall  f1-score   support

           0       0.87      0.99      0.93      6493
           1       0.89      0.29      0.44      1372

    accuracy                           0.87      7865
   macro avg       0.88      0.64      0.68      7865
weighted avg       0.87      0.87      0.84      7865

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7838
           1       0.00      0.00      0.00        27

    accuracy                           1.00      7865
   macro avg       0.50      0.50      0.50      786

c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.95      1.00      0.98      7481
           1       0.71      0.04      0.08       384

    accuracy                           0.95      7865
   macro avg       0.83      0.52      0.53      7865
weighted avg       0.94      0.95      0.93      7865

search_and_rescue
              precision    recall  f1-score   support

           0       0.97      1.00      0.99      7649
           1       0.00      0.00      0.00       216

    accuracy                           0.97      7865
   macro avg       0.49      0.50      0.49      7865
weighted avg       0.95      0.97      0.96      7865

security
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7727
           1       0.00      0.00      0.00       138

    accuracy                           0.98      7865
   macro avg       0.49      0.50      0.50      7865
weighted avg       0.97      0.98      0.97    

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7689
           1       0.83      0.03      0.05       176

    accuracy                           0.98      7865
   macro avg       0.91      0.51      0.52      7865
weighted avg       0.97      0.98      0.97      7865

other_weather
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      7463
           1       0.75      0.02      0.04       402

    accuracy                           0.95      7865
   macro avg       0.85      0.51      0.51      7865
weighted avg       0.94      0.95      0.93      7865

direct_report
              precision    recall  f1-score   support

           0       0.83      0.99      0.91      6285
           1       0.87      0.21      0.33      1580

    accuracy                           0.83      7865
   macro avg       0.85      0.60      0.62      7865
weighted avg       0.84      0.83      0.79   

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [16]:
#Improve  the pipeline
pipeline2 = Pipeline([
    ('vect', CountVectorizer()),
    ('best', TruncatedSVD()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [17]:
pipeline2.fit(X_train, y_train)
perf_report(pipeline2, X_test, y_test)

related
              precision    recall  f1-score   support

           0       0.38      0.00      0.01      1813
           1       0.76      1.00      0.87      5995
           2       0.80      0.14      0.24        57

    accuracy                           0.76      7865
   macro avg       0.65      0.38      0.37      7865
weighted avg       0.68      0.76      0.66      7865

request
              precision    recall  f1-score   support

           0       0.83      1.00      0.90      6493
           1       0.00      0.00      0.00      1372

    accuracy                           0.83      7865
   macro avg       0.41      0.50      0.45      7865
weighted avg       0.68      0.83      0.75      7865

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7838
           1       0.00      0.00      0.00        27

    accuracy                           1.00      7865
   macro avg       0.50      0.50      0.50      786

c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.97      1.00      0.99      7649
           1       0.00      0.00      0.00       216

    accuracy                           0.97      7865
   macro avg       0.49      0.50      0.49      7865
weighted avg       0.95      0.97      0.96      7865

security
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7727
           1       0.00      0.00      0.00       138

    accuracy                           0.98      7865
   macro avg       0.49      0.50      0.50      7865
weighted avg       0.97      0.98      0.97      7865

military
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      7597
           1       0.00      0.00      0.00       268

    accuracy                           0.97      7865
   macro avg       0.48      0.50      0.49      7865
weighted avg       0.93      0.97      0.95      7865

c

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7689
           1       0.00      0.00      0.00       176

    accuracy                           0.98      7865
   macro avg       0.49      0.50      0.49      7865
weighted avg       0.96      0.98      0.97      7865

other_weather
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      7463
           1       0.00      0.00      0.00       402

    accuracy                           0.95      7865
   macro avg       0.47      0.50      0.49      7865
weighted avg       0.90      0.95      0.92      7865

direct_report
              precision    recall  f1-score   support

           0       0.80      1.00      0.89      6285
           1       0.00      0.00      0.00      1580

    accuracy                           0.80      7865
   macro avg       0.40      0.50      0.44      7865
weighted avg       0.64      0.80      0.71   

In [18]:
#Param tunning 
parameters2 = { #'vect__ngram_range': ((1, 1), (1, 2)), 
              #'vect__max_df': (0.5, 1.0), 
              #'vect__max_features': (None, 5000), 
              'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [50, 100],
              'clf__estimator__learning_rate': [1,2] }

In [19]:
cv2 = GridSearchCV(pipeline2, param_grid=parameters2)
cv2

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

In [20]:
cv2.fit(X_train, y_train)

c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

In [21]:
perf_report(cv2, X_test, y_test)

related
              precision    recall  f1-score   support

           0       0.50      0.01      0.01      1813
           1       0.76      1.00      0.87      5995
           2       0.67      0.14      0.23        57

    accuracy                           0.76      7865
   macro avg       0.64      0.38      0.37      7865
weighted avg       0.70      0.76      0.66      7865

request
              precision    recall  f1-score   support

           0       0.83      1.00      0.90      6493
           1       0.00      0.00      0.00      1372

    accuracy                           0.83      7865
   macro avg       0.41      0.50      0.45      7865
weighted avg       0.68      0.83      0.75      7865

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7838
           1       0.00      0.00      0.00        27

    accuracy                           1.00      7865
   macro avg       0.50      0.50      0.50      786

c:\users\ahmad\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.97      1.00      0.99      7649
           1       0.00      0.00      0.00       216

    accuracy                           0.97      7865
   macro avg       0.49      0.50      0.49      7865
weighted avg       0.95      0.97      0.96      7865

security
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7727
           1       0.00      0.00      0.00       138

    accuracy                           0.98      7865
   macro avg       0.49      0.50      0.50      7865
weighted avg       0.97      0.98      0.97      7865

military
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      7597
           1       0.00      0.00      0.00       268

    accuracy                           0.97      7865
   macro avg       0.48      0.50      0.49      7865
weighted avg       0.93      0.97      0.95      7865

c

              precision    recall  f1-score   support

           0       0.80      1.00      0.89      6285
           1       0.00      0.00      0.00      1580

    accuracy                           0.80      7865
   macro avg       0.40      0.50      0.44      7865
weighted avg       0.64      0.80      0.71      7865



### 9. Export your model as a pickle file

In [22]:
with open('model.pkl', 'wb') as f:
    pickle.dump(cv2, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.